In [2]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
import os
from urllib.request import urlretrieve
from openrouteservice.directions import directions
from openrouteservice import convert
import operator
from functools import reduce
import openrouteservice as ors
import math


# Functions
def getClient(key):
    return ors.Client(key=key)


def getKmDistance(locations):
    distance = 0
    for i in range(len(locations) - 1):
        distance += haversine(locations[i][0], locations[i][1], locations[i + 1][0], locations[i + 1][1])
    return distance


# Find the stright line distance between two lat lons
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


def getRoute(client, loc1, loc2): # loc template => (lon, lat)
    # white house to the pentagon
    coords = [loc1, loc2]

    route = client.directions(coordinates=coords,
                            profile='driving-car',
                            format="geojson")
    
    return route


# Find the route distance between two lat lons
def findRouteDist(lat1, lat2, lon1, lon2):
    # key = "5b3ce3597851110001cf624806471ffca1c34a59affc7b8696347bee"

    # client = getClient(key)
    client = ors.Client(base_url="http://localhost:8080/ors")

    loc_1 = (lon1, lat1)
    loc_2 = (lon2, lat2)

    route = getRoute(client, loc_1, loc_2)

    locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]

    distance = getKmDistance(locations)

    return distance

In [17]:
findRouteDist(lat1 = -37.773160, lon1=145.082079, lat2=-37.806386, lon2=145.175529)

12.240073164966057

In [7]:
import requests

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
}

lat1 = -37.773160
lon1=145.082079
lat2=-37.806386
lon2=145.175529

call = requests.get(f'http://localhost:8080/ors/v2/directions/driving-car?start={lon1},{lat1}&end={lon2},{lat2}', headers=headers)

print(call.status_code, call.reason)
print(call.text)

200 
{"type":"FeatureCollection","metadata":{"attribution":"openrouteservice.org, OpenStreetMap contributors, tmc - BASt","service":"routing","timestamp":1695293772662,"query":{"coordinates":[[145.082079,-37.77316],[145.175529,-37.806386]],"profile":"driving-car","format":"json"},"engine":{"version":"8.0","build_date":"2023-09-20T07:54:24Z","graph_date":"2023-09-20T11:27:39Z"}},"bbox":[145.078659,-37.809496,145.178134,-37.770562],"features":[{"bbox":[145.078659,-37.809496,145.178134,-37.770562],"type":"Feature","properties":{"transfers":0,"fare":0,"segments":[{"distance":12240.1,"duration":762.8,"steps":[{"distance":443.4,"duration":83.5,"type":11,"instruction":"Head northwest","name":"-","way_points":[0,12]},{"distance":956.3,"duration":90.8,"type":0,"instruction":"Turn left onto Bulleen Road, 52","name":"Bulleen Road, 52","way_points":[12,22]},{"distance":288.5,"duration":28.9,"type":12,"instruction":"Keep left onto Thompsons Road","name":"Thompsons Road","way_points":[22,29]},{"dist

In [3]:
df = pd.read_csv('property.csv')

In [16]:
df['coordinates'][0]

'[-37.8176878 144.9597256]'

In [12]:
from shapely import wkt

# df['closestTrain'] = df['closestTrain'].apply(wkt.loads)
print(type(df['closestTrain'][0]))

<class 'shapely.geometry.point.Point'>


In [17]:
import ast

def convertString(input_string):
    # Remove the square brackets and split the string by whitespace
    split_string = input_string.strip('[]').split()

    # Convert the elements to numbers if needed
    result_list = [float(x) for x in split_string]
    return result_list


df['coordinates'] = df['coordinates'].apply(convertString)

In [19]:
print((df['coordinates'][0]))

[-37.8176878, 144.9597256]


In [3]:
client = ors.Client(base_url="http://localhost:8080/ors")

In [7]:
loc_1 = (144.9610166, -37.8089811)
loc_2 = (144.9671, -37.8183)
coords = [loc_1, loc_2]
route = client.directions(coordinates=coords,
                            profile='driving-car',
                            format="geojson")

In [19]:
route

{'type': 'FeatureCollection',
 'metadata': {'attribution': 'openrouteservice.org, OpenStreetMap contributors, tmc - BASt',
  'service': 'routing',
  'timestamp': 1696571853225,
  'query': {'coordinates': [[144.9610166, -37.8089811], [144.9671, -37.8183]],
   'profile': 'driving-car',
   'format': 'geojson'},
  'engine': {'version': '8.0',
   'build_date': '2023-09-20T07:54:24Z',
   'graph_date': '2023-09-20T11:27:39Z'}},
 'bbox': [144.960848, -37.823369, 144.970214, -37.809029],
 'features': [{'bbox': [144.960848, -37.823369, 144.970214, -37.809029],
   'type': 'Feature',
   'properties': {'transfers': 0,
    'fare': 0,
    'segments': [{'distance': 3102.0,
      'duration': 460.2,
      'steps': [{'distance': 375.7,
        'duration': 63.1,
        'type': 11,
        'instruction': 'Head south on Elizabeth Street',
        'name': 'Elizabeth Street',
        'way_points': [0, 10]},
       {'distance': 463.4,
        'duration': 56.5,
        'type': 0,
        'instruction': 'Turn l

In [31]:
(route['features'])

[{'bbox': [144.960848, -37.823369, 144.970214, -37.809029],
  'type': 'Feature',
  'properties': {'transfers': 0,
   'fare': 0,
   'segments': [{'distance': 3102.0,
     'duration': 460.2,
     'steps': [{'distance': 375.7,
       'duration': 63.1,
       'type': 11,
       'instruction': 'Head south on Elizabeth Street',
       'name': 'Elizabeth Street',
       'way_points': [0, 10]},
      {'distance': 463.4,
       'duration': 56.5,
       'type': 0,
       'instruction': 'Turn left onto Lonsdale Street',
       'name': 'Lonsdale Street',
       'way_points': [10, 23]},
      {'distance': 711.2,
       'duration': 99.3,
       'type': 1,
       'instruction': 'Turn right onto Russell Street',
       'name': 'Russell Street',
       'way_points': [23, 43]},
      {'distance': 231.5,
       'duration': 34.5,
       'type': 1,
       'instruction': 'Turn right onto Flinders Street, 30',
       'name': 'Flinders Street, 30',
       'way_points': [43, 47]},
      {'distance': 366.9,
   

In [47]:
(route['features'][0]['properties']['segments'][0]['distance'])

3102.0